In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# -----------------------------
# Load dataset
# -----------------------------
df = pd.read_csv("data.csv")

# Keep only relevant features
df = df[['Product_Category','Customer_Segment','Shipping_Method','Payment_Method',
         'Gender','Income','Total_Purchases']]

# Create classification target: High Purchaser = 1 if purchases >= 3
df['High_Purchase'] = (df['Total_Purchases'] >= 3).astype(int)

# Drop original Total_Purchases
df = df.drop(columns=['Total_Purchases'])

# Split features & target
X = df.drop(columns=['High_Purchase'])
y = df['High_Purchase']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert categorical to category dtype
categorical_cols = ['Product_Category','Customer_Segment','Shipping_Method','Payment_Method','Gender','Income']
for col in categorical_cols:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

# Train LightGBM classifier
model = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=10,
    num_leaves=64,
    random_state=42
)
model.fit(X_train, y_train, categorical_feature=categorical_cols)

# -----------------------------
# Evaluate
# -----------------------------
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("✅ Model trained to classify High Purchasers")
print(f"Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec*100:.2f}%")
print(f"Recall: {rec*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")

# -----------------------------
# Prediction function
# -----------------------------
def predict(product_category, customer_segment, shipping_method, payment_method, gender, income):
    row = pd.DataFrame([[product_category, customer_segment, shipping_method, payment_method,
                         gender, income]],
                       columns=X.columns)
    for col in categorical_cols:
        row[col] = row[col].astype('category')
    pred = model.predict(row)[0]
    if pred == 1:
        return "🟢 Likely High Purchaser → No urgent promotion needed."
    else:
        return "🔴 Low Purchaser → Consider promotion."

# -----------------------------
# Gradio interface
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 🛒 Promotion Recommendation System")
    gr.Markdown(f"**Model Performance:** Accuracy = {acc*100:.2f}%, Precision = {prec*100:.2f}%, Recall = {rec*100:.2f}%, F1 = {f1*100:.2f}%")

    with gr.Row():
        product_input = gr.Dropdown(sorted(df['Product_Category'].unique()), label="Product Category")
        segment_input = gr.Dropdown(sorted(df['Customer_Segment'].unique()), label="Customer Segment")

    with gr.Row():
        shipping_input = gr.Dropdown(sorted(df['Shipping_Method'].unique()), label="Shipping Method")
        payment_input = gr.Dropdown(sorted(df['Payment_Method'].unique()), label="Payment Method")

    with gr.Row():
        gender_input = gr.Dropdown(sorted(df['Gender'].unique()), label="Gender")
        income_input = gr.Dropdown(sorted(df['Income'].unique()), label="Income Level")

    output = gr.Textbox(label="Promotion Recommendation")

    gr.Button("Predict").click(
        fn=predict,
        inputs=[product_input, segment_input, shipping_input, payment_input, gender_input, income_input],
        outputs=output
    )

demo.launch(share=True)
